# Actverse Analysis

## 1. actverse analysis 설치

In [ ]:
"""colab setup"""
! pip install actverse@git+https://github.com/actnova-inc/actverse-analysis

"""local setup"""
# ! pip install ../../

"""common setup"""
! pip install matplotlib

## 1. actverse 결과 로드
actverse-web으로 부터 얻은 prediction url 또는 로컬 파일의 경로를 입력

In [ ]:
from actverse.utils import load_json

json_path = input("Downloadable url or local file path:")

prediction = load_json(json_path)

## 2. 분석

actverse-web부터 얻은 결과로 분석 수행


In [ ]:
from actverse.analysis import measure_physical_metrics

metrics, ids = measure_physical_metrics(prediction)

## 3. 시각화

이동 metrics 시각화 - 속력, 누적 이동 거리, 평균 속도 등

In [ ]:
img_width = prediction["metadata"]["origin_width"]
img_height = prediction["metadata"]["origin_height"]

In [ ]:
import matplotlib.pyplot as plt

from actverse.utils import moving_average

mouse_id = ids[0]
plt.figure(f"movement-{mouse_id}", figsize=(10, 10))
plt.subplots_adjust(hspace=0.5)
plt.subplots_adjust(wspace=0.25)

n_rows, n_cols = 3, 2
plt.subplot(n_rows, n_cols, 1)
plt.title("Distance Moved")
plt.plot(
    metrics[mouse_id]["timestamp"], metrics[mouse_id]["cumulative_distance_change"]
)
plt.ylabel("Distance (pixel)")
plt.xlabel("Time (s)")

plt.subplot(n_rows, n_cols, 2)
plt.title("Speed")
plt.plot(metrics[mouse_id]["timestamp"], metrics[mouse_id]["speed"])
plt.ylabel("Speed (pixel/s)")
plt.xlabel("Time (s)")

plt.subplot(n_rows, n_cols, 3)
plt.title("Speed (Smoothed)")
smoothed_speed = moving_average(metrics[mouse_id]["speed"], 25)
plt.plot(metrics[mouse_id]["timestamp"], smoothed_speed)
plt.ylabel("Speed (pixel/s)")
plt.xlabel("Time (s)")

plt.subplot(n_rows, n_cols, 4)
plt.title("Speed (histogram)")
plt.hist(metrics[mouse_id]["speed"], bins=50)
plt.xlabel("Speed (pixel/s)")
plt.ylabel("Frequency")

plt.subplot(n_rows, n_cols, 5)
plt.title("Average Speed")
plt.plot(metrics[mouse_id]["timestamp"], metrics[mouse_id]["average_speed"])
plt.ylabel("Speed (pixel/s)")
plt.xlabel("Time (s)")


plt.show()

방향 metrics 시각화 - 각도, 누적 각도 변화, 각속도

In [ ]:
mouse_id = ids[0]
plt.figure(f"angle-{mouse_id}", figsize=(10, 10))
plt.subplots_adjust(hspace=0.5)
plt.subplots_adjust(wspace=0.25)

n_rows, n_cols = 3, 2
plt.subplot(n_rows, n_cols, 1)
plt.title("Body Angle")
plt.plot(metrics[mouse_id]["timestamp"], metrics[mouse_id]["angle"])
plt.ylabel("Angle (degree)")
plt.xlabel("Time (s)")

plt.subplot(n_rows, n_cols, 2)
plt.title("Angular Speed")
plt.plot(metrics[mouse_id]["timestamp"], metrics[mouse_id]["angular_speed"])
plt.ylabel("Angular Speed (degree/s)")
plt.xlabel("Time (s)")

plt.subplot(n_rows, n_cols, 3)
plt.title("Body Angle (histogram)")
plt.hist(metrics[mouse_id]["angle"], bins=50)
plt.xlabel("Angle (degree)")
plt.ylabel("Frequency")

plt.subplot(n_rows, n_cols, 4)
plt.title("Angular Speed (histogram)")
plt.hist(metrics[mouse_id]["angular_speed"], bins=50)
plt.xlabel("Angular Speed (degree/s)")
plt.ylabel("Frequency")

plt.subplot(n_rows, n_cols, 5)
plt.title("Angular Speed (Smoothed)")
smoothed_angular_speed = moving_average(metrics[mouse_id]["angular_speed"], 25)
plt.plot(metrics[mouse_id]["timestamp"], smoothed_angular_speed)
plt.ylabel("Angular Speed (degree/s)")
plt.xlabel("Time (s)")

위치 metrics 시각화 - trajectory 및 히트맵

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_lines(points, cmap_name="plasma"):
    cmap = plt.get_cmap(cmap_name, len(points))  # 컬러맵 설정

    for i in range(len(points) - 1):
        x0, y0 = points[i]
        x1, y1 = points[i + 1]
        color = cmap(i)
        plt.plot([x0, x1], [y0, y1], linestyle="-", color=color)


mouse_id = ids[0]
plt.figure(f"position-{mouse_id}", figsize=(10, 5))

n_rows, n_cols = 1, 2
plt.subplot(n_rows, n_cols, 1)
plt.title("Trajectory")

plot_lines(metrics[mouse_id]["position"])
plt.xlabel("X (pixel)")
plt.ylabel("Y (pixel)")

plt.subplot(n_rows, n_cols, 2)
plt.title("Heatmap")
positions = np.array(metrics[mouse_id]["position"])
plt.hist2d(
    positions[:, 0],
    positions[:, 1],
    bins=50,
    cmap="hot",
)
plt.xlabel("X (pixel)")
plt.ylabel("Y (pixel)")

plt.show()